#### naver 영화검색
>영진회에서 얻은 영화명(국문)을 바탕으로 네이버 영화 검색 API를 통해 추가적인 데이터를 수집합니다.
* 요청
- 영화명을 통해 요청합니다.

* 응답
- 영화별로 다음과 같은 내용을 저장합니다. 영진위 영화 대표코드, 영화 썸네일 이미지의 URL, 하이퍼텍스트 link, 유저 평점
- 해당 결과를 movie_naver.csv에 저장합니다.

#### 영화 포스터 이미지 저장
> 네이버 영화 검색 API를 통해 얻은 이미지 URL에 요청을 보내 실제 이미지 파일로 저장합니다.
* 요청
- 이미지 URL
* 응답
- 응답받은 결과를 파일로 저장합니다. 저장시 반드시 wb옵션으로 저장하시기 바랍니다.
- 저장되는 파일명은 images폴더 내에 영진위 영화 대표코드.jpg입니다.

### 결과
> 결과물은 반드시 README.md 로 활용했던 API정보를 정리하고, 결과로 저장된 csv파일에 대한 설명을 기록해야 합니다.
* movie/
* README.md
* *.py
* boxoffice.csv
* movie.csv
* movie_naver.csv
* images/
* *.jpg

In [ ]:
#image가 두세개 없으니 그냥 둬도 되는데 random 이미지를 넣거나 default값을 넣어주면 됨

#네이버는 for문 막 돌리면 막아버리기 때문에 잠시 쉬어줘야함
from time import sleep
sleep(0.05)

In [ ]:
import requests

movies = ['말모이', '주먹왕 랄프', '보헤미안 랩소디']

naver_uri = 'https://openapi.naver.com/v1/search/movie.json?query='

client_id = 
client_secret = 
headers = { 
    'X-Naver-Client-Id': client_id, 
    'X-Naver-Client-Secret': client_secret
}
res = requests.get(naver_uri + movies[0], headers=headers)
data = res.json()

In [ ]:
result = []
for movie in movies:
    data_set = requests.get(naver_uri + movie, headers=headers).json()
    movie_info = {}
    movie_info['title'] = data_set['items'][0]['title']
    movie_info['link'] = data_set['items'][0]['link']
    movie_info['image'] = data_set['items'][0]['image']
    movie_info['userRating'] = data_set['items'][0]['userRating']
    result.append(movie_info)

In [ ]:
result
for i in result:
    print(i.values())

In [ ]:
import os
import csv
import re
import requests
from datetime import datetime, timedelta

KOBIS_KEY = os.getenv('KOBIS_KEY')
NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')

def targetDt_date(start_y, start_m, start_d):
    re_list = []
    time1 = datetime(start_y, start_m, start_d)
    for i in range(10):
        time2 = time1 + timedelta(weeks=i)
        result = int(time2.strftime('%Y%m%d'))
        re_list.append(result)
    return re_list

def total_data():
    targetDt = targetDt_date(2018,11,11)
    key = KOBIS_KEY
    
    tot_dict = {}
    for target in targetDt:
        URL = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&weekGb=0&targetDt={target}'
        data = requests.get(URL).json()
        data = data['boxOfficeResult']['weeklyBoxOfficeList']
        
        for ranking in range(10):
            code_data = data[ranking]['movieCd']
            title_data = data[ranking]['movieNm']
            audience_data = data[ranking]['audiAcc']
            tot_dict[code_data] = [code_data, title_data, audience_data, target]
    return tot_dict

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def naver():
    naver_uri = 'https://openapi.naver.com/v1/search/movie.json?query='
    client_id = NAVER_CLIENT_ID
    client_secret = NAVER_CLIENT_SECRET
    headers = { 
        'X-Naver-Client-Id': client_id, 
        'X-Naver-Client-Secret': client_secret
    }
    
    result_dict = {}
    pair = {}
    for code, name in total_data().items():
        pair[code] = name[1]

    for naver_code, naver_name in pair.items():
        data_set = requests.get(naver_uri + naver_name, headers=headers).json()
        title = cleanhtml(data_set['items'][0]['title'])
        link = data_set['items'][0]['link']
        image = data_set['items'][0]['image']
        userRating = data_set['items'][0]['userRating']
        result_dict[naver_name] = [naver_code, image, link, userRating]
    return result_dict
    

naver_list = open('movie_naver.csv', 'a+', encoding='utf-8', newline='')
writer = csv.writer(naver_list) 
naver = naver()
for navers in naver.values():
    writer.writerow(navers)
naver_list.close() 

import os
from PIL import Image
import requests



In [ ]:
def pair():
    pairs = {}
    for code, name in total_data().items():
        pairs[code] = name[1]
    return pairs

In [ ]:
import os
import csv
import re
import requests
from datetime import datetime, timedelta

KOBIS_KEY = os.getenv('KOBIS_KEY')
NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')

def targetDt_date(start_y, start_m, start_d):
    re_list = []
    time1 = datetime(start_y, start_m, start_d)
    for i in range(10):
        time2 = time1 + timedelta(weeks=i)
        result = int(time2.strftime('%Y%m%d'))
        re_list.append(result)
    return re_list

def total_data():
    targetDt = targetDt_date(2018,11,11)
    key = KOBIS_KEY
    
    tot_dict = {}
    for target in targetDt:
        URL = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&weekGb=0&targetDt={target}'
        data = requests.get(URL).json()
        data = data['boxOfficeResult']['weeklyBoxOfficeList']
        
        for ranking in range(10):
            code_data = data[ranking]['movieCd']
            title_data = data[ranking]['movieNm']
            audience_data = data[ranking]['audiAcc']
            tot_dict[code_data] = [code_data, title_data, audience_data, target]
    return tot_dict

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    
def pairs():
    pair = {}
    for code, name in total_data().items():
        pair[code] = name[1]
    return pair

def naver():
    naver_uri = 'https://openapi.naver.com/v1/search/movie.json?query='
    client_id = NAVER_CLIENT_ID
    client_secret = NAVER_CLIENT_SECRET
    headers = { 
        'X-Naver-Client-Id': client_id, 
        'X-Naver-Client-Secret': client_secret
    }
    result_dict = {}
    for naver_code, naver_name in pairs().items():
        data_set = requests.get(naver_uri + naver_name, headers=headers).json()
        title = cleanhtml(data_set['items'][0]['title'])
        link = data_set['items'][0]['link']
        image = data_set['items'][0]['image']
        userRating = data_set['items'][0]['userRating']
        result_dict[naver_name] = [naver_code, image, link, userRating]
    return result_dict

naver_list = open('movie_naver.csv', 'a+', encoding='utf-8', newline='')
writer = csv.writer(naver_list) 
for na in naver().values():
    writer.writerow(na)
naver_list.close() 

#
for save_img in naver().values():
    code = save_img[0]
    image_url = save_img[1]
    image = requests.get(image_url).content  
    filename = f'{code}.jpg'   
with open(filename, 'wb') as f: 
    f.write(image) 

In [ ]:
import os
from PIL import Image
import requests

for save_img in naver().values():
    code = save_img[0]
    image_url = save_img[1]
    image = requests.get(image_url).content  #서버 응답을 받아 파일내용 획득. content는 응답받은 RawData
    filename = f'{code}.jpg'   #URL에서 파일명 획득. 뒷부분의 python.png란 파일명만 저장
with open(filename, 'wb') as f:  #wb: 쓰기 파이러니
    f.write(image) #파일 저장